In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
import seaborn as sns
import pickle

In [ ]:
!unzip /content/268833_611395_compressed_AB_NYC_2019.csv.zip

unzip:  cannot find or open /content/268833_611395_compressed_AB_NYC_2019.csv.zip, /content/268833_611395_compressed_AB_NYC_2019.csv.zip.zip or /content/268833_611395_compressed_AB_NYC_2019.csv.zip.ZIP.


In [ ]:
df = pd.read_csv('AB_NYC_2019.csv')

In [ ]:
df2 = pd.read_csv('listings.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43,61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df2 = df2[['id', 'accommodates']]

In [ ]:
df2 = df2.dropna()

In [ ]:
df.drop('host_name', axis=1, inplace=True)
df.drop('name', axis=1, inplace=True)
df.drop('last_review', axis=1, inplace=True)
df.drop('calculated_host_listings_count', axis=1, inplace=True)
df.drop('reviews_per_month', axis=1, inplace=True)
df.drop('latitude', axis=1, inplace=True)
df.drop('longitude', axis=1, inplace=True)
df.drop('number_of_reviews', axis=1, inplace=True)
df.drop('availability_365', axis=1, inplace=True)
df.drop('neighbourhood_group', axis=1, inplace=True)

In [ ]:
df = df[df['price'] < 1001]

In [ ]:
df = df[df['price'] > 24]

In [ ]:
new_df = df.merge(df2, on='id')

In [ ]:
df = pd.DataFrame(new_df)

In [ ]:
df.drop('id', axis=1, inplace=True)
df.drop('host_id', axis=1, inplace=True)

In [ ]:
#room_type, minimum_nights, accommodates, neighbourhood

#Private room, Entire home/apt, Shared room

#a number

#a number

#neighbourhood values below
df.neighbourhood.unique()

array(['Midtown', 'Clinton Hill', 'Murray Hill', 'Bedford-Stuyvesant',
       "Hell's Kitchen", 'Upper West Side', 'Chinatown', 'South Slope',
       'West Village', 'Williamsburg', 'Fort Greene', 'East Harlem',
       'Inwood', 'East Village', 'Harlem', 'Bushwick', 'Flatbush',
       'Lower East Side', 'Prospect-Lefferts Gardens', 'Long Island City',
       'Greenpoint', 'Kips Bay', 'SoHo', 'Chelsea', 'Upper East Side',
       'Prospect Heights', 'Park Slope', 'Washington Heights', 'Woodside',
       'Brooklyn Heights', 'Carroll Gardens', 'Gowanus', 'Flatlands',
       'Cobble Hill', 'Flushing', 'Boerum Hill', 'Sunnyside',
       'St. George', 'Highbridge', 'Ridgewood', 'Morningside Heights',
       'Jamaica', 'Middle Village', 'Ditmars Steinway',
       'Flatiron District', 'Windsor Terrace', 'Roosevelt Island',
       'Greenwich Village', 'Little Italy', 'Tompkinsville', 'Astoria',
       'Kensington', 'Clason Point', 'Eastchester', 'Crown Heights',
       'Kingsbridge', 'Queens Vil

In [ ]:
### BASELINE ###

df['price'].mean()

137.4559556624245

In [ ]:
# Encode data

encoder = OneHotEncoder()
label_encoder = LabelEncoder()

In [ ]:
words = ['room_type', 'neighbourhood']

In [ ]:
df.head()

,neighbourhood,room_type,price,minimum_nights,accommodates
0,Midtown,Entire home/apt,225,1,2
1,Clinton Hill,Entire home/apt,89,1,3
2,Murray Hill,Entire home/apt,200,3,2
3,Bedford-Stuyvesant,Private room,60,45,2
4,Hell's Kitchen,Private room,79,2,2


In [ ]:
for col in words:
  df[col] = label_encoder.fit_transform(df[col])
  df[col] = df[col].astype(int)

In [ ]:
df.head()

,neighbourhood,room_type,price,minimum_nights,accommodates
0,126,0,225,1,2
1,41,0,89,1,3
2,136,0,200,3,2
3,13,1,60,45,2
4,94,1,79,2,2


In [ ]:
train, test = train_test_split(df, test_size=0.045, random_state=42)

print(f'train: {train.shape}')
print(f'test: {test.shape}\n')

X_train = train.drop(columns='price')
y_train = train['price']
X_test = test.drop(columns='price')
y_test = test['price']

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

train: (32567, 5)
test: (1535, 5)

X_train shape: (32567, 4)
y_train shape: (32567,)
X_test shape: (1535, 4)
y_test shape: (1535,)


In [ ]:
df.head()

,neighbourhood,room_type,price,minimum_nights,accommodates
0,126,0,225,1,2
1,41,0,89,1,3
2,136,0,200,3,2
3,13,1,60,45,2
4,94,1,79,2,2


In [ ]:
# Normalize the data

sc = StandardScaler()
X = sc.fit_transform(X_train)

In [ ]:
# Model

inputs = X.shape[1]

model = Sequential()

# Add to model
#18, 12, 4, 1
#adam, mae

model.add(Dense(32, activation='relu', input_dim=inputs))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1),)

# Compile model
model.compile(loss='mae', optimizer='RMSprop', metrics=['mse'])

# Fit model
model.fit(X, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
1018/1018 [==============================] - 3s 3ms/step - loss: 72.8138 - mae: 72.8138
Epoch 2/10
1018/1018 [==============================] - 3s 3ms/step - loss: 51.1478 - mae: 51.1478
Epoch 3/10
1018/1018 [==============================] - 3s 3ms/step - loss: 50.6265 - mae: 50.6265
Epoch 4/10
1018/1018 [==============================] - 3s 3ms/step - loss: 50.3814 - mae: 50.3814
Epoch 5/10
1018/1018 [==============================] - 3s 3ms/step - loss: 50.2293 - mae: 50.2293
Epoch 6/10
1018/1018 [==============================] - 3s 3ms/step - loss: 50.1299 - mae: 50.1299
Epoch 7/10
1018/1018 [==============================] - 3s 3ms/step - loss: 50.0745 - mae: 50.0745
Epoch 8/10
1018/1018 [==============================] - 3s 3ms/step - loss: 50.0216 - mae: 50.0216
Epoch 9/10
1018/1018 [==============================] - 3s 2ms/step - loss: 49.9986 - mae: 49.9986
Epoch 10/10
1018/1018 [==============================] - 3s 3ms/step - loss: 49.9827 - mae: 49.9827


In [ ]:
#use utils for preprocess, pickle encoders?, pipeline for prepro., 

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                160       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 833
Trainable params: 833
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import dill
from sklearn.externals import joblib
'''filename = open('mmodel2.pkl','wb')
pickle.dump(model,filename)
filename.close()'''
#with open('model_pickled.pkl','wb') as file:
    #dill.dump(model,file)
path = "/content/nyc_model"
model.save('model.h5')

In [ ]:
from tensorflow.keras.models import load_model
m = load_model('model.h5')

In [ ]:
#model.save("nyc_model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: nyc_model/assets


In [ ]:
#reconstructed_model = keras.models.load_model("nyc_model")

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#np.testing.assert_allclose(
#    model.predict(X_test), reconstructed_model.predict(X_test)
#)


NameError: ignored

In [ ]:
row1 = df.iloc[[0]]

In [ ]:
row1

,neighbourhood,room_type,price,minimum_nights,accommodates
0,126,0,225,1,2


In [ ]:
row1.drop('price', axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
m.predict(row1)

array([[2855.6106]], dtype=float32)

In [ ]:
#reconstructed_model.predict(row1)

array([[3250.185]], dtype=float32)

In [ ]:
### LINEAR REGRESSION/RANDOM FOREST WORK ###

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(OneHotEncoder(categories=words),
                         RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, criterion='mse'))
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(categories='auto', drop=None,
                               dtype=<class 'numpy.float64'>,
                               handle_unknown='error', sparse=True)),
                ('randomforestregressor',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=None,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=100, n_jobs=-1,
                                       oob_score=False, random_state=42,
         

In [ ]:
print('Validation Accuracy', pipeline.score(X_val, y_val))

NameError: ignored

In [ ]:
df = pd.read_csv('AB_NYC_2019.csv')

In [ ]:
### VISUALIZATIONS ###
import matplotlib.image as mpimg

sub_500 = df[df.price < 500]

In [ ]:
heat_map = sub_500.plot(kind='scatter', x='longitude', y='latitude',
                        label='availability_365', c='price',
                        cmap=plt.get_cmap('jet'), colorbar=True, alpha=0.4,
                        figsize=(10, 8))

In [ ]:
# Heat map w/ NYC picture

import urllib

plt.figure(figsize=(10,8))

i=urllib.request.urlopen('https://upload.wikimedia.org/wikipedia/commons/e/ec/Neighbourhoods_New_York_City_Map.PNG')
nyc_img=plt.imread(i)

plt.imshow(nyc_img,zorder=0,extent=[-74.258, -73.7, 40.49,40.92])
ax=plt.gca()


sub_500.plot(kind='scatter', x='longitude', y='latitude', label='availability_365', c='price', ax=ax, 
           cmap=plt.get_cmap('jet'), colorbar=True, alpha=0.4, zorder=5)
plt.legend()
plt.show()

In [ ]:
# TASKS:

# 3) Continue to tune model

# 6) Run GridSearch, random forest or something else on model